In [2]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import math
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset,random_split

In [3]:
import os
os.chdir("./../.")
os.getcwd()

'/Users/namihosono/OneDrive - Universität Mannheim/AnimationSVG/deepsvg-master'

In [3]:
torch.autograd.set_detect_anomaly(True)

In [6]:
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimationSVG/deepsvg-master/AnimateSVG_logo/data/embeddings/path_embedding.pkl", "rb") as f:
    inp = pickle.load(f)

with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimateSVG_2/data/animated_svgs_dataframes/1505_animation_vectors.pkl", "rb") as f:
    output = pickle.load(f)

output = output[output['file'].str.contains('animation_0')][["file","model_output"]]

names = output["file"].str.replace("_animation_0", "")

inp = inp[inp["filename"].isin(names)]

filenames = inp["filename"].unique()
#print(filenames)
list = []
for name in filenames:
    seq = inp[inp["filename"]==name].loc[:, ~inp.columns.isin(["filename","animation_id"])][:4]
    seq = pd.concat([seq, pd.DataFrame(10, index=seq.index, columns=range(256, 268))], axis=1, ignore_index=True)

    
    sos = pd.DataFrame([[30]*268])
    
    eos = pd.DataFrame([[50]*268])
    

    seq = pd.concat([sos, seq, eos], ignore_index=True)

    #seq.loc[len(seq)] = [999] * len(seq.columns)
    #seq.loc[-1] = [-999] * len(seq.columns)
    #seq.index = seq.index + 1
    #seq.sort_index(inplace=True)
    #print(seq)
    list.append(torch.tensor(seq.values))
inpTensor = torch.stack(list)
print(inpTensor.shape)

torch.Size([401, 6, 268])


In [7]:
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimationSVG/deepsvg-master/AnimateSVG_logo/data/embeddings/path_embedding.pkl", "rb") as f:
    inp = pickle.load(f)
inp

,filename,animation_id,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,logo_0,1,0.851117,-1.775123,0.649689,-0.688600,0.216071,0.135211,-1.748761,-1.347670,...,-1.616431,-1.572003,0.242460,0.430259,0.079752,-1.039526,-0.696104,0.090277,-0.228757,0.144372
1,logo_0,0,0.763518,-0.982797,-0.446681,1.089468,-0.070563,0.710206,-0.491675,-1.631172,...,0.339061,0.022934,0.195161,-0.046488,-0.492103,-0.605836,-1.282879,0.613195,0.297194,-0.172312
2,logo_0,2,0.328628,-0.612160,1.592824,-0.422598,-0.765523,1.140459,0.145881,0.240300,...,-0.254159,-0.790789,-0.057158,1.594653,0.395605,-1.048015,0.454987,-0.439861,1.430237,0.192277
3,logo_0,3,0.291136,-0.928242,0.265542,-0.261439,-0.386160,1.256256,-0.414706,-1.206105,...,-0.446020,-1.369758,0.356421,1.456656,0.468766,-1.077724,-0.548627,-0.300660,0.632805,-0.136473
4,logo_0,4,0.504446,-0.543099,0.915062,1.293575,-0.849605,1.120387,-0.637641,-1.337280,...,0.162753,0.206993,-0.201259,-1.087391,-0.597388,-0.992079,-0.851486,-0.225463,-0.549269,0.088637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10804,logo_9,3,0.341127,-0.844122,0.851799,-0.352970,-0.859344,0.568410,-1.287803,-0.416381,...,-1.519855,-4.074135,-0.355853,1.073493,1.398292,-1.681473,-1.260963,-0.233300,-0.015946,0.974122
10805,logo_9,6,0.424217,-1.353261,1.238835,-0.837542,-0.345199,-0.974526,-1.683693,-0.507383,...,-3.137928,-3.240320,0.277688,-0.590543,0.265793,-1.681521,-0.566898,0.476020,-0.685451,1.096683
10806,logo_9,7,1.531518,-1.612310,0.801471,1.137218,0.327335,1.749122,-0.426959,-0.576917,...,1.142574,-1.827454,-0.635960,2.076138,0.755637,-0.566260,-0.688139,-0.606805,0.903784,-0.709454
10807,logo_9,5,0.621977,-2.617885,1.425599,-0.207073,0.257930,-0.073098,-2.433338,-0.637954,...,-2.948898,-3.191816,0.459644,0.794539,0.686447,-1.263997,-0.288715,0.684191,-0.894395,0.695853


In [8]:
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimateSVG_2/data/animated_svgs_dataframes/1505_animation_vectors.pkl", "rb") as f:
    output = pickle.load(f)
model_output = output[output['file'].str.contains('animation_0')][["file","model_output"]]

filenames = model_output["file"].unique()
list = []
for name in filenames:
    seq = model_output[model_output["file"]==name]
    seq = seq["model_output"]
    seq = pd.DataFrame(output["model_output"].to_list(), columns=["a1","a2","a3","a4","a5","a6","a7","a8","a9","a10","a11","a12"])
    
    seq = pd.concat([pd.DataFrame(10, index=seq.index, columns=range(0, 256)), seq], axis=1, ignore_index=True)
    
    if len(seq) > 4:
        seq = seq[:4]

    sos = pd.DataFrame([[30]*268])
    
    eos = pd.DataFrame([[50]*268])

    seq = pd.concat([sos, seq, eos], ignore_index=True)
    
    while len(seq) < 6:
           seq = pd.concat([seq, pd.DataFrame([[-100]*268])], ignore_index=True)
           
    #seq = seq.apply(lambda x: np.array(x).astype(np.float32))
    #tokens = []
    #for l in seq:
    #    tokens.append(torch.tensor(l))

    list.append(torch.tensor(seq.values))
outTensor = torch.stack(list)
print(outTensor.shape)

torch.Size([401, 6, 268])


In [26]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [38]:
class TransformerEmbedded(nn.Module):
    """
    Model from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    # Constructor
    def __init__(
        self,
        num_tokens,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        # INFO
        self.model_type = "TransformerEmbedded"
        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=5000
        )
        self.embedding = nn.Embedding(num_tokens, dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
            batch_first=True
        )
        self.out = nn.Linear(dim_model, num_tokens)
        
    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):
        # Src size must be (batch_size, src sequence length)
        # Tgt size must be (batch_size, tgt sequence length)

        # Embedding + positional encoding - Out size = (batch_size, sequence length, dim_model)
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)
        
        # We could use the parameter batch_first=True, but our KDL version doesn't support it yet, so we permute
        # to obtain size (sequence length, batch_size, dim_model),
        #src = src.permute(1,0,2)
        #tgt = tgt.permute(1,0,2)

        # Transformer blocks - Out size = (sequence length, batch_size, num_tokens)
        transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        
        out = self.out(transformer_out)
        
        return out
      
    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask

    def create_pad_mask(self, input : torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]

        t = []
        for seq in input:
            list =[]
            for token in seq:
                b = False
                for value in token:
                    if value != pad_token:
                        b = True
                list.append(b)
            t.append(list)
        return torch.tensor(t)

In [39]:

model = TransformerEmbedded(
    num_tokens=268, dim_model=268, num_heads=4, num_encoder_layers=4, num_decoder_layers=4, dropout_p=0.1
)
opt = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [45]:
print(model)

TransformerEmbedded(
  (positional_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embedding): Embedding(268, 268)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=268, out_features=268, bias=True)
          )
          (linear1): Linear(in_features=268, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=268, bias=True)
          (norm1): LayerNorm((268,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((268,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj

### Working Transformer

In [59]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        batch_size = Q.size()[0]
        seq_length = Q.size()[1]
        #mask = mask.view(batch_size, 1, 1, seq_length).expand(-1, self.num_heads, -1, -1)
        mask = mask.unsqueeze(1).unsqueeze(1).expand(-1, self.num_heads, -1, -1)
        mask = mask.permute(0, 1, 3, 2)
        #print("mask shape:",mask.shape)
        #print(attn_scores.shape)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == True, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        #print("Q:",Q.shape)
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [60]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):

        #d_ff: dimension of hidden layer

        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [61]:
class PositionalEncoding2(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding2, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [62]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [63]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [64]:
class TransformerFromscratch(nn.Module):
    def __init__(self, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(TransformerFromscratch, self).__init__()
        #self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        #self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding2(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 10).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 10).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        print("seq_length",seq_length)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        print("nopeak:",nopeak_mask.shape)
        print("tgt_mask:",tgt_mask.shape)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask
    
    def create_pad_mask(self, input : torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]

        t = []
        for seq in input:
            list =[]
            for token in seq:
                b = False
                for value in token:
                    if value == pad_token:
                        b = True
                list.append(b)
            t.append(list)
        return torch.tensor(t)

        #return (input == pad_token)
    
    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a square matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask


    def forward(self, src, tgt):
        #src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_mask = self.create_pad_mask(src,pad_token=10)
        tgt_mask = self.create_pad_mask(tgt,pad_token=10)
        src_embedded = self.dropout(self.positional_encoding(src))
        tgt_embedded = self.dropout(self.positional_encoding(tgt))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [65]:
model2 = TransformerFromscratch(tgt_vocab_size=268, d_model=268, num_heads=4, num_layers=8, d_ff=2048, dropout=0.1,max_seq_length=10)
opt = torch.optim.SGD(model2.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

In [303]:
scr_m = model2.create_pad_mask(inpTensor,10)
scr_m

tensor([[False,  True,  True,  True,  True, False],
        [False,  True,  True,  True,  True, False],
        [False,  True,  True,  True,  True, False],
        ...,
        [False,  True,  True,  True,  True, False],
        [False,  True,  True,  True,  True, False],
        [False,  True,  True,  True,  True, False]])

In [132]:
scr_m.shape

torch.Size([401, 6])

In [66]:
def train_loop(model, opt, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.train()
    total_loss = 0
    
    
    for batch in dataloader:
        X, y = batch[0], batch[1]
        #X, y = torch.tensor(X), torch.tensor(y)

        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        y_input = y[:,:-1]
        y_expected = y[:,1:]
        
        # Get mask to mask out the next words
        #sequence_length = y.size(1)
        #tgt_mask = model.get_tgt_mask(sequence_length)
        
        #print(y.view(y.size(0), -1).shape)
        #pad_mask = model.create_pad_mask(y, pad_token=-100)
        #print(pad_mask)

        # Standard training except we pass in y_input and tgt_mask
        
        #print(torch.isnan(X).any())
        #print(torch.isnan(y).any())

        #pred = model(X, y, tgt_mask, pad_mask)

        pred = model2(X,y)

        #print("pred shape:",pred.shape)
        #print("y shape:",y.shape)

        y_flattened = y.contiguous().view(-1)

        #print(y_flattened)

        # Permute pred to have batch size first again
        #pred = pred.permute(1, 2, 0)      
        loss = loss_fn(pred, y)

        #print("prediction:",pred)

        opt.zero_grad()
        loss.backward()

        # tryign to print the gradient
        #for p in model.parameters():
        #    print(p.grad.norm())

        #for name, param in model2.named_parameters():
        #    if 'weight' in name:
         #       print(name)
         #       print(param.data.cpu().numpy().shape)
         #       print('gradient is \t', param.grad, '\trequires grad: ', param.requires_grad)

        # gradient clipping to avoid the exploding gradient problem
        torch.nn.utils.clip_grad_value_(model2.parameters(), 10.)

        opt.step()
    
        total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

In [67]:
from torch.utils.data import TensorDataset, DataLoader
train = TensorDataset(inpTensor2.float(), outTensor2.float())
batch_size = 120 # Set your desired batch size
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)  # For input data

In [69]:
print(torch.isnan(inpTensor2).any())
print(torch.isnan(outTensor2).any())

tensor(False)
tensor(False)


In [70]:
def fit(model, opt, loss_fn, train_dataloader, epochs):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    # Used for plotting later on
    train_loss_list, validation_loss_list = [], []
    
    print("Training and validating model")
    for epoch in range(epochs):
        print("-"*25, f"Epoch {epoch + 1}","-"*25)
        
        train_loss = train_loop(model2, opt, loss_fn, train_dataloader)
        train_loss_list += [train_loss]
        
        #validation_loss = validation_loop(model, loss_fn, val_dataloader)
        #validation_loss_list += [validation_loss]
        
        print(f"Training loss: {train_loss:.4f}")
        #print(f"Validation loss: {validation_loss:.4f}")
        print()
        
    return train_loss_list, validation_loss_list
    
train_loss_list, validation_loss_list = fit(model2, opt, loss_fn, train_dataloader, 10)

Training and validating model
------------------------- Epoch 1 -------------------------
Training loss: 613.9722

------------------------- Epoch 2 -------------------------
Training loss: 569.1013

------------------------- Epoch 3 -------------------------
Training loss: 531.6607

------------------------- Epoch 4 -------------------------
Training loss: 488.3676

------------------------- Epoch 5 -------------------------
Training loss: 446.4761

------------------------- Epoch 6 -------------------------
Training loss: 400.9464

------------------------- Epoch 7 -------------------------
Training loss: 351.6340

------------------------- Epoch 8 -------------------------
Training loss: 305.9851

------------------------- Epoch 9 -------------------------
Training loss: 269.6919

------------------------- Epoch 10 -------------------------
Training loss: 249.2559



In [309]:
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimationSVG/deepsvg-master/AnimateSVG_logo/data/surrogate_model/animation_label.pkl", "rb") as f:
    surrogate = pickle.load(f)

In [298]:
surrogate

,file,animation_id,order_id,path_prob,begin_value,model_output,animated_animation_ids,animated_order_ids,backend_mapping,logo_id,animation_number,alias,animation_file,time,label
0,logo_316_animation_0,4,1,0.200000,1.00,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
1,logo_316_animation_0,24,6,0.600000,1.25,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
2,logo_316_animation_0,23,7,0.200000,1.50,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
3,logo_316_animation_1,4,1,0.200000,1.00,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Very Bad
4,logo_316_animation_1,24,6,0.600000,1.25,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Okay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23781,logo_91_animation_22,22,1,0.916667,1.00,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[21, 22]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",91,22,NaN,NaN,NaN,NaN
23782,logo_91_animation_23,21,0,0.916667,1.25,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[21, 22]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",91,23,NaN,NaN,NaN,NaN
23783,logo_91_animation_23,22,1,0.916667,1.00,"[0, 1, 0, 0, 0, 0, -1.0, -1.0, 0.9248652516259...","[21, 22]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",91,23,NaN,NaN,NaN,NaN
23784,logo_91_animation_24,21,0,0.916667,1.25,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[21, 22]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",91,24,NaN,NaN,NaN,NaN


In [18]:
print(surrogate['label'].value_counts())

Okay         4621
Very Bad     3040
Bad          2531
Good         2398
no_rating    1303
Very Good     509
Name: label, dtype: int64


In [19]:
print(surrogate['file'].value_counts())

logo_379_animation_5     14
logo_334_animation_12    10
logo_347_animation_14     8
logo_121_animation_18     8
logo_86_animation_8       8
                         ..
logo_277_animation_16     1
logo_277_animation_15     1
logo_289_animation_4      1
logo_277_animation_10     1
logo_371_animation_17     1
Name: file, Length: 9907, dtype: int64


In [20]:
print(surrogate['alias'].value_counts())

Jonathan     5385
Tim          4622
Jakob        1868
Rebecca       704
Sarah         698
JONATHAN      450
Ole           340
Niklas         90
jani           80
TEST           66
Jani           55
Cocorina       17
jOHNATHAN       6
Name: alias, dtype: int64


In [22]:
#getting only the goos and very good values
target_values = ['Good','Very Good']

subset = surrogate[surrogate['label'].isin(target_values)]

subset

,file,animation_id,order_id,path_prob,begin_value,model_output,animated_animation_ids,animated_order_ids,backend_mapping,logo_id,animation_number,alias,animation_file,time,label
0,logo_316_animation_0,4,1,0.200000,1.00,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
1,logo_316_animation_0,24,6,0.600000,1.25,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
2,logo_316_animation_0,23,7,0.200000,1.50,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
12,logo_316_animation_4,1,0,0.200000,1.25,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.2...","[1, 4, 2, 24, 23]","[0, 1, 4, 6, 7]","[1, 1, 0, 0, 1, 0, 1, 1]",316,4,Jani,animation/logo_316_animation_4.svg,"Timestamp(seconds=1617889079, nanoseconds=2880...",Good
13,logo_316_animation_4,4,1,0.600000,1.00,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[1, 4, 2, 24, 23]","[0, 1, 4, 6, 7]","[1, 1, 0, 0, 1, 0, 1, 1]",316,4,Jani,animation/logo_316_animation_4.svg,"Timestamp(seconds=1617889079, nanoseconds=2880...",Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23728,logo_27_animation_15,134,0,1.000000,1.00,"[0, 1, 0, 0, 0, 0, -1.0, -1.0, 0.9652421415521...",[134],[0],"[1, 0, 0, 0, 0, 0, 0, 0]",27,15,Tim,animation/logo_27_animation_15.svg,"Timestamp(seconds=1618767024, nanoseconds=5000...",Good
23746,logo_91_animation_4,21,0,0.916667,1.25,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.2...","[21, 22]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",91,4,Rebecca,animation/logo_91_animation_4.svg,"Timestamp(seconds=1617889500, nanoseconds=4800...",Good
23748,logo_91_animation_5,21,0,0.916667,1.25,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[21, 22]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",91,5,Jakob,animation/logo_91_animation_5.svg,"Timestamp(seconds=1617894446, nanoseconds=4100...",Good
23749,logo_91_animation_5,22,1,0.916667,1.00,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[21, 22]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",91,5,Jakob,animation/logo_91_animation_5.svg,"Timestamp(seconds=1617894446, nanoseconds=4100...",Good


In [23]:
print(subset['file'].value_counts())

logo_431_animation_1     7
logo_347_animation_12    7
logo_80_animation_2      6
logo_110_animation_5     6
logo_379_animation_2     6
                        ..
logo_56_animation_4      1
logo_265_animation_11    1
logo_213_animation_6     1
logo_213_animation_5     1
logo_91_animation_13     1
Name: file, Length: 1887, dtype: int64


In [13]:
print(pd.__version__)

1.3.5


In [ ]:
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimateSVG_2/data/animated_svgs_dataframes/1505_animation_vectors.pkl", "rb") as f:
    whole_output = pickle.load(f)
whole_output

,file,animation_id,order_id,path_prob,begin_value,model_output,animated_animation_ids,animated_order_ids,backend_mapping
0,logo_317_animation_0,30,0,1.0,1.00,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[30, 31, 32]","[0, 1, 2]","[1, 1, 1, 0, 0, 0, 0, 0]"
1,logo_317_animation_0,31,1,1.0,1.25,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...","[30, 31, 32]","[0, 1, 2]","[1, 1, 1, 0, 0, 0, 0, 0]"
2,logo_317_animation_0,32,2,0.8,1.50,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.4...","[30, 31, 32]","[0, 1, 2]","[1, 1, 1, 0, 0, 0, 0, 0]"
3,logo_317_animation_1,30,0,1.0,1.00,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[30, 31]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]"
4,logo_317_animation_1,31,1,1.0,1.25,"[0, 1, 0, 0, 0, 0, -1.0, -1.0, 0.1343642441124...","[30, 31]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...
4716,logo_332_animation_2,25,1,1.0,1.00,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.0565513...","[24, 25]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]"
4717,logo_332_animation_3,24,0,1.0,1.25,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[24, 25]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]"
4718,logo_332_animation_3,25,1,1.0,1.00,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.2...","[24, 25]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]"
4719,logo_332_animation_4,24,0,1.0,1.25,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.2...","[24, 25]","[0, 1]","[1, 1, 0, 0, 0, 0, 0, 0]"


In [ ]:
surrogate[["file","model_output"]].head()

,file,model_output
0,logo_316_animation_0,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8..."
1,logo_316_animation_0,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1...."
2,logo_316_animation_0,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1...."
3,logo_316_animation_1,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474..."
4,logo_316_animation_1,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746..."


### filtering the output

In [18]:
# get the file with ratings
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimationSVG/deepsvg-master/AnimateSVG_logo/data/surrogate_model/animation_label.pkl", "rb") as f:
    surrogate = pickle.load(f)

#Filter with only good or very good ratings
#filtered_output = surrogate[surrogate)][['label'].isin(['Good','Very Good']["file","animation_id","model_output","label"]]
filtered_output = surrogate[["file","animation_id","model_output","label"]]

# get the names of unique logos by splitting with animation number
logos = filtered_output["file"].str.split("_animation").str[0].unique()

#print(logos)

# create a data frame for the collected best animations
bestoutput = pd.DataFrame()

# go through each logo to find the best animation
for logo in logos:

    # make a data frame that contains all the animations of one logo
    temp = filtered_output[filtered_output["file"].str.contains(logo)]

    #print("temp",temp)
    
    # see if it has very good or not and take the first one
    if temp["label"].eq("Very Good").any():
        
        bestlogo = temp[temp["label"] == "Very Good"].iloc[0][["file"]]
    else:
        
        bestlogo = temp.iloc[0][["file"]]
    
    # get all the animated paths with the best animation of the logo
    best_animations = temp[temp["file"]==bestlogo["file"]]

    # add to the file
    bestoutput = pd.concat([bestoutput,best_animations],axis=0, ignore_index=True)


#bestOutput=bestOutput.transpose()
display(bestoutput)

,file,animation_id,model_output,label
0,logo_316_animation_15,1,"[0, 1, 0, 0, 0, 0, -1.0, -1.0, 0.9652421415521...",no_rating
1,logo_316_animation_15,24,"[1, 0, 0, 0, 0, 0, 0.011654693792141124, 0.735...",Very Good
2,logo_316_animation_15,23,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",Okay
3,logo_378_animation_2,28,"[1, 0, 0, 0, 0, 0, 0.9560342718892494, 0.94782...",Good
4,logo_378_animation_2,29,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.0565513...",Very Good
...,...,...,...,...
1019,logo_262_animation_0,1,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Good
1020,logo_361_animation_1,15,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Very Good
1021,logo_279_animation_2,25,"[1, 0, 0, 0, 0, 0, 0.9560342718892494, 0.94782...",Very Good
1022,logo_91_animation_0,21,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Okay


In [19]:
print(bestoutput['label'].value_counts())

Very Good    356
Good         232
Bad          143
Okay         124
Very Bad     113
no_rating     18
Name: label, dtype: int64


### Filtering the output 2

In [57]:
# get the file with ratings
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimationSVG/deepsvg-master/AnimateSVG_logo/data/surrogate_model/animation_label.pkl", "rb") as f:
    surrogate2 = pickle.load(f)

#Filter with only good or very good ratings
#filtered_output = surrogate[surrogate)][['label'].isin(['Good','Very Good']["file","animation_id","model_output","label"]]
filtered_output2 = surrogate2[["file","animation_id","model_output","label"]]

# dictionary for mapping

mapping_dict = {"Very Good": 6, "Good": 5, "Bad": 4,"Okay":3, "Very Bad": 2, "no_rating": 1}

# Create another column changing the label into ints

filtered_output2['label2'] = filtered_output2['label'].replace(mapping_dict)


# get the names of unique logos by splitting with animation number
logos = filtered_output2["file"].str.split("_animation").str[0].unique()

#print(logos)

# create a data frame for the collected best animations
bestoutput2 = pd.DataFrame()

# go through each logo to find the best animation
for logo in logos:

    # make a data frame that contains all the animations of one logo
    temp = filtered_output2[filtered_output2["file"].str.contains(logo)]

    #display(temp)

    # create a sum 
    mean_by_label = temp.groupby('file')['label2'].mean().reset_index()

    #print(mean_by_label)

    bestlogo = mean_by_label.loc[mean_by_label['label2'].idxmax()]

    #print(bestlogo)

    # get all the animated paths with the best animation of the logo
    best_animations2 = temp[temp["file"]==bestlogo["file"]]

    # add to the file
    bestoutput2 = pd.concat([bestoutput2,best_animations2],axis=0, ignore_index=True)
bestoutput2 = bestoutput2.sort_values(by=['file','animation_id'])
display(bestoutput2)

filenames = bestoutput2["file"].unique()
list = []
for name in filenames:
    seq = bestoutput2[bestoutput2["file"]==name]
    seq = seq["model_output"]
    seq = pd.DataFrame(bestoutput2["model_output"].to_list(), columns=["a1","a2","a3","a4","a5","a6","a7","a8","a9","a10","a11","a12"])
    
    seq = pd.concat([pd.DataFrame(10, index=seq.index, columns=range(0, 256)), seq], axis=1, ignore_index=True)
    
    if len(seq) > 4:
        seq = seq[:4]

    sos = pd.DataFrame([[30]*268])
    
    eos = pd.DataFrame([[50]*268])

    seq = pd.concat([sos, seq, eos], ignore_index=True)
    
    while len(seq) < 6:
           seq = pd.concat([seq, pd.DataFrame([[-100]*268])], ignore_index=True)
           
    #seq = seq.apply(lambda x: np.array(x).astype(np.float32))
    #tokens = []
    #for l in seq:
    #    tokens.append(torch.tensor(l))

    list.append(torch.tensor(seq.values))
outTensor2 = torch.stack(list)
print(outTensor2.shape)

/Users/namihosono/anaconda3/envs/deepsvg/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,file,animation_id,model_output,label,label2
436,logo_0_animation_0,0,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Good,5.0
439,logo_0_animation_0,1,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0
438,logo_0_animation_0,3,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0
437,logo_0_animation_0,4,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.4205715...",Good,5.0
435,logo_0_animation_0,5,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0
...,...,...,...,...,...
892,logo_99_animation_0,21,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.4...",Very Good,6.0
811,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Very Good,6.0
891,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Very Good,6.0
810,logo_99_animation_0,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Very Good,6.0


torch.Size([359, 6, 268])


### Creating the input tensor with the diltered output

In [58]:
with open("/Users/namihosono/OneDrive - Universität Mannheim/AnimationSVG/deepsvg-master/AnimateSVG_logo/data/embeddings/path_embedding.pkl", "rb") as f:
    inp = pickle.load(f)

bestoutput2["filename"] = bestoutput2["file"].str.split("_animation").str[0]

display(bestoutput2)

inp['animation_id'] = inp['animation_id'].astype(int)

#names = bestoutput2["file"].str.replace("_animation_0", "")

#input = inp[(inp["filename"].isin(bestoutput2["filename"])) & (inp["animation_id"].isin(bestoutput2["animation_id"]))]
input = pd.merge(bestoutput2, inp, on=['filename', 'animation_id'],how='inner')
input = input.drop(['model_output', 'label','label2','file'], axis=1)

input = input.sort_values(by=['filename','animation_id'])
display(input)
filenames = input["filename"].unique()
#print(filenames)
list = []
for name in filenames:
    #print(name)
    seq = input[input["filename"]==name].loc[:, ~inp.columns.isin(["filename","animation_id"])][:4]
    #print(seq)
    seq = pd.concat([seq, pd.DataFrame(10, index=seq.index, columns=range(256, 268))], axis=1, ignore_index=True)

    while len(seq) < 4:
        seq = pd.concat([seq, pd.DataFrame([[10]*268])], ignore_index=True)

    sos = pd.DataFrame([[30]*268])

    
    eos = pd.DataFrame([[50]*268])

    seq = pd.concat([sos, seq, eos], ignore_index=True)

    list.append(torch.tensor(seq.values))
    #print(list)
inpTensor2 = torch.stack(list)
print(inpTensor2.shape)

,file,animation_id,model_output,label,label2,filename
436,logo_0_animation_0,0,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Good,5.0,logo_0
439,logo_0_animation_0,1,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0,logo_0
438,logo_0_animation_0,3,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0,logo_0
437,logo_0_animation_0,4,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.4205715...",Good,5.0,logo_0
435,logo_0_animation_0,5,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0,logo_0
...,...,...,...,...,...,...
892,logo_99_animation_0,21,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.4...",Very Good,6.0,logo_99
811,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Very Good,6.0,logo_99
891,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Very Good,6.0,logo_99
810,logo_99_animation_0,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Very Good,6.0,logo_99


,animation_id,filename,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,0,logo_0,0.763518,-0.982797,-0.446681,1.089468,-0.070563,0.710206,-0.491675,-1.631172,...,0.339061,0.022934,0.195161,-0.046488,-0.492103,-0.605836,-1.282879,0.613195,0.297194,-0.172312
1,1,logo_0,0.851117,-1.775123,0.649689,-0.688600,0.216071,0.135211,-1.748761,-1.347670,...,-1.616431,-1.572003,0.242460,0.430259,0.079752,-1.039526,-0.696104,0.090277,-0.228757,0.144372
2,3,logo_0,0.291136,-0.928242,0.265542,-0.261439,-0.386160,1.256256,-0.414706,-1.206105,...,-0.446020,-1.369758,0.356421,1.456656,0.468766,-1.077724,-0.548627,-0.300660,0.632805,-0.136473
3,4,logo_0,0.504446,-0.543099,0.915062,1.293575,-0.849605,1.120387,-0.637641,-1.337280,...,0.162753,0.206993,-0.201259,-1.087391,-0.597388,-0.992079,-0.851486,-0.225463,-0.549269,0.088637
4,5,logo_0,-0.641569,-0.657125,-0.105109,-0.031630,-0.572032,0.912017,-0.569627,-1.573482,...,-1.079276,0.285482,1.532865,-0.375210,-0.249130,-0.551393,-1.024246,0.623726,-1.073305,0.166613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,21,logo_99,0.729620,-1.035586,0.390709,1.910684,-0.489203,2.197111,0.424709,-0.389426,...,1.130544,-1.892641,-0.690288,1.371062,0.718980,-0.827454,-0.557172,0.235162,0.523978,-0.592003
903,22,logo_99,1.147938,-0.799806,0.664411,1.610482,-0.612273,1.194499,0.076532,-0.662109,...,0.415811,-1.569571,-0.424470,0.518902,0.160598,-1.283986,-0.463894,0.319661,0.785325,-0.294870
904,22,logo_99,1.147938,-0.799806,0.664411,1.610482,-0.612273,1.194499,0.076532,-0.662109,...,0.415811,-1.569571,-0.424470,0.518902,0.160598,-1.283986,-0.463894,0.319661,0.785325,-0.294870
905,23,logo_99,1.645128,-1.911424,0.806118,0.850210,0.554424,1.578238,-0.722733,-0.704658,...,0.883759,-2.000007,-0.517095,2.213172,0.733567,-0.577074,-0.710312,-0.591997,0.836300,-0.720247


torch.Size([359, 6, 268])


In [36]:
bestoutput2_sort = bestoutput2.sort_values(by=['filename','animation_id'])

display(bestoutput2_sort)

,file,animation_id,model_output,label,label2,filename
436,logo_0_animation_0,0,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Good,5.0,logo_0
439,logo_0_animation_0,1,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0,logo_0
438,logo_0_animation_0,3,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0,logo_0
437,logo_0_animation_0,4,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.4205715...",Good,5.0,logo_0
435,logo_0_animation_0,5,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0,logo_0
...,...,...,...,...,...,...
892,logo_99_animation_0,21,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.4...",Very Good,6.0,logo_99
811,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Very Good,6.0,logo_99
891,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",Very Good,6.0,logo_99
810,logo_99_animation_0,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Very Good,6.0,logo_99


In [54]:
bestoutput2[bestoutput2["filename"].str.contains("logo_100")]

,file,animation_id,model_output,label,label2,filename
701,logo_100_animation_12,0,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",Good,5.0,logo_100


In [40]:
input[input["filename"].str.contains("logo_0")]

,filename,animation_id,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
1,logo_0,0,0.763518,-0.982797,-0.446681,1.089468,-0.070563,0.710206,-0.491675,-1.631172,...,0.339061,0.022934,0.195161,-0.046488,-0.492103,-0.605836,-1.282879,0.613195,0.297194,-0.172312
0,logo_0,1,0.851117,-1.775123,0.649689,-0.688600,0.216071,0.135211,-1.748761,-1.347670,...,-1.616431,-1.572003,0.242460,0.430259,0.079752,-1.039526,-0.696104,0.090277,-0.228757,0.144372
2,logo_0,2,0.328628,-0.612160,1.592824,-0.422598,-0.765523,1.140459,0.145881,0.240300,...,-0.254159,-0.790789,-0.057158,1.594653,0.395605,-1.048015,0.454987,-0.439861,1.430237,0.192277
3,logo_0,3,0.291136,-0.928242,0.265542,-0.261439,-0.386160,1.256256,-0.414706,-1.206105,...,-0.446020,-1.369758,0.356421,1.456656,0.468766,-1.077724,-0.548627,-0.300660,0.632805,-0.136473
4,logo_0,4,0.504446,-0.543099,0.915062,1.293575,-0.849605,1.120387,-0.637641,-1.337280,...,0.162753,0.206993,-0.201259,-1.087391,-0.597388,-0.992079,-0.851486,-0.225463,-0.549269,0.088637
5,logo_0,5,-0.641569,-0.657125,-0.105109,-0.031630,-0.572032,0.912017,-0.569627,-1.573482,...,-1.079276,0.285482,1.532865,-0.375210,-0.249130,-0.551393,-1.024246,0.623726,-1.073305,0.166613
8,logo_0,6,1.063729,-1.362633,1.147410,-1.257667,-0.022295,0.515886,-0.932486,0.251040,...,-1.183255,-4.326854,-0.377295,3.353210,1.904817,-1.396564,-0.330079,-0.740591,1.867344,0.399258
6,logo_0,7,1.089030,-1.687915,1.293008,-1.140435,0.162678,0.299779,-1.249240,0.119432,...,-1.664449,-4.577971,-0.474324,3.010473,1.694227,-1.456383,-0.382898,-0.484506,1.530320,0.347399
7,logo_0,8,1.158221,-1.272248,1.707672,-1.483284,-0.106444,-0.471404,-1.344110,0.346676,...,-1.824265,-4.101873,-0.425078,2.203583,1.257311,-1.554118,-0.194875,-0.824977,1.708750,0.906707
9,logo_0,9,0.914504,-1.837203,1.429079,-1.977881,0.191471,-0.501183,-1.765532,0.074680,...,-2.660991,-4.542898,-0.018555,2.336879,1.480729,-1.581533,-0.184397,-0.349435,1.007312,0.780156


In [29]:
bestoutput2.dtypes

file             object
animation_id      int64
model_output     object
label            object
label2          float64
filename         object
dtype: object

In [30]:
inp.dtypes

filename         object
animation_id     object
0               float32
1               float32
2               float32
                 ...   
251             float32
252             float32
253             float32
254             float32
255             float32
Length: 258, dtype: object

In [17]:
print(bestoutput2['label'].value_counts())

Good         393
Very Good    355
Bad          115
Okay          27
Very Bad      17
Name: label, dtype: int64


In [14]:
surrogate2[surrogate2["file"].str.contains("logo_316")]

,file,animation_id,order_id,path_prob,begin_value,model_output,animated_animation_ids,animated_order_ids,backend_mapping,logo_id,animation_number,alias,animation_file,time,label
0,logo_316_animation_0,4,1,0.2,1.00,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
1,logo_316_animation_0,24,6,0.6,1.25,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
2,logo_316_animation_0,23,7,0.2,1.50,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
3,logo_316_animation_1,4,1,0.2,1.00,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Very Bad
4,logo_316_animation_1,24,6,0.6,1.25,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Okay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,logo_316_animation_24,1,0,0.2,1.25,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[1, 4, 2, 24, 23]","[0, 1, 4, 6, 7]","[1, 1, 0, 0, 1, 0, 1, 1]",316,24,NaN,NaN,NaN,NaN
83,logo_316_animation_24,4,1,0.6,1.00,"[0, 1, 0, 0, 0, 0, -1.0, -1.0, 0.7123429878269...","[1, 4, 2, 24, 23]","[0, 1, 4, 6, 7]","[1, 1, 0, 0, 1, 0, 1, 1]",316,24,NaN,NaN,NaN,NaN
84,logo_316_animation_24,2,4,0.2,1.50,"[0, 1, 0, 0, 0, 0, -1.0, -1.0, 0.8397997363118...","[1, 4, 2, 24, 23]","[0, 1, 4, 6, 7]","[1, 1, 0, 0, 1, 0, 1, 1]",316,24,NaN,NaN,NaN,NaN
85,logo_316_animation_24,24,6,0.2,1.75,"[0, 1, 0, 0, 0, 0, -1.0, -1.0, 0.1825918869545...","[1, 4, 2, 24, 23]","[0, 1, 4, 6, 7]","[1, 1, 0, 0, 1, 0, 1, 1]",316,24,NaN,NaN,NaN,NaN


In [5]:
bestoutput['label'].value_counts()

Very Good    356
Good         232
Bad          143
Okay         124
Very Bad     113
no_rating     18
Name: label, dtype: int64

In [ ]:
model_output = subset[["file","model_output"]]

filenames = model_output["file"].unique()
list = []
for name in filenames:
    seq = model_output[model_output["file"]==name]
    seq = seq["model_output"]
    seq = pd.DataFrame(output["model_output"].to_list(), columns=["a1","a2","a3","a4","a5","a6","a7","a8","a9","a10","a11","a12"])
    
    seq = pd.concat([pd.DataFrame(0, index=seq.index, columns=range(0, 256)), seq], axis=1, ignore_index=True)
    
    if len(seq) > 4:
        seq = seq[:4]

    sos = pd.DataFrame([[-999]*12])
    
    eos = pd.DataFrame([[999]*12])

    seq = pd.concat([sos, seq, eos], ignore_index=True)
    
    while len(seq) < 6:
           seq = pd.concat([seq, pd.DataFrame([[-100]*12])], ignore_index=True)
           
    #seq = seq.apply(lambda x: np.array(x).astype(np.float32))
    #tokens = []
    #for l in seq:
    #    tokens.append(torch.tensor(l))

    list.append(torch.tensor(seq.values))
outTensor = torch.stack(list)
print(outTensor.shape)

torch.Size([1887, 6, 268])


In [ ]:
#with open(path_to_embedding, "rb") as f:
    inp = pickle.load(f)

#print(inp)

#with open(path_to_vectors, "rb") as f:
    output = pickle.load(f)
#output = output[output['file'].str.contains('animation_0')][["file","model_output"]]

output = output[output["label"].isin(["Good", "Very Good"])][["file","animation_id","model_output","label"]]

logos = output["file"].str.split("_animation").str[0]

bestOutput = pd.DataFrame()
for logo in logos:
    temp = output[output["file"].str.contains(logo)]
    if temp["label"].eq("Very Good").any():
        bestOutput = pd.concat([bestOutput, pd.DataFrame(temp[temp["label"] == "Very Good"].iloc[0])], axis=1, ignore_index=True)
    else:
        bestOutput = pd.concat([bestOutput, pd.DataFrame(temp.iloc[0])], axis=1, ignore_index=True)
bestOutput=bestOutput.transpose()
print(bestOutput)